In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir samples

In [ ]:
!pip install tensorboardX==1.4

     |████████████████████████████████| 67 kB 3.0 MB/s 


In [ ]:
import torch
import torch.nn
import argparse
import logging
from functools import wraps

FORMAT = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
LEVEL = logging.DEBUG
logging.basicConfig(format=FORMAT, level=LEVEL)
log = logging.getLogger(__name__)
log.info('Entered module: %s' % __name__)


def logger(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        log = logging.getLogger(fn.__name__)
        log.info('Start running %s' % fn.__name__)

        out = fn(*args, **kwargs)

        log.info('Done running %s' % fn.__name__)
        # Return the return value
        return out

    return wrapper


def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def weight_init(m):
    if isinstance(m, torch.nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)


2021-08-20 09:37:49,384 - __main__ - INFO - Entered module: __main__


In [ ]:
# from utils import str2bool
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--ori_data_path', type=str, default='ori',  help='Origin image path')
parser.add_argument('--haze_data_path', type=str, default='haze',  help='Haze image path')
parser.add_argument('--val_ori_data_path', type=str, default='val_ori',  help='Validation origin image path')
parser.add_argument('--val_haze_data_path', type=str, default='val_haze',  help='Validation haze image path')
parser.add_argument('--sample_output_folder', type=str, default='samples',  help='Validation haze image path')
parser.add_argument('--use_gpu', type=str2bool, default=True, help='Use GPU')
parser.add_argument('--gpu', type=int, default=-1, help='GPU id')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning Rate. Default=1e-4')
parser.add_argument('--num_workers', type=int, default=4, help='Number of threads for data loader, for window set to 0')
parser.add_argument('--print_gap', type=int, default=50, help='number of batches to print average loss ')
parser.add_argument('--batch_size', type=int, default=16, help='Training batch size')
parser.add_argument('--val_batch_size', type=int, default=16, help='Validation batch size')
parser.add_argument('--epochs', type=int, default=10, help='number of epochs for training')
parser.add_argument('--model_dir', type=str, default='./model')
parser.add_argument('--log_dir', type=str, default='./log')
parser.add_argument('--ckpt', type=str, default='./model/nets/net_1.pkl')
parser.add_argument('--net_name', type=str, default='nets')
parser.add_argument('--weight_decay', type=float, default=0.0001)
parser.add_argument('--grad_clip_norm', type=float, default=0.1)


def get_config():
    config, unparsed = parser.parse_known_args()
    return config, unparsed


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AODnet(nn.Module):
    def __init__(self):
        super(AODnet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=6, out_channels=3, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(in_channels=6, out_channels=3, kernel_size=7, stride=1, padding=3)
        self.conv5 = nn.Conv2d(in_channels=12, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.b = 1

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = F.relu(self.conv2(x1))
        cat1 = torch.cat((x1, x2), 1)
        x3 = F.relu(self.conv3(cat1))
        cat2 = torch.cat((x2, x3), 1)
        x4 = F.relu(self.conv4(cat2))
        cat3 = torch.cat((x1, x2, x3, x4), 1)
        k = F.relu(self.conv5(cat3))

        if k.size() != x.size():
            raise Exception("k, haze image are different size!")

        output = k * x - k + self.b
        return F.relu(output)


In [ ]:
import os
import torch
from PIL import Image
import glob
import random


class HazeDataset(torch.utils.data.Dataset):
    def __init__(self, ori_root, haze_root, transforms):
        self.haze_root = haze_root
        self.ori_root = ori_root
        self.image_name_list = glob.glob(os.path.join(self.haze_root, '*.jpg'))
        self.matching_dict = {}
        self.file_list = []
        self.get_image_pair_list()
        self.transforms = transforms
        print("Total data examples:", len(self.file_list))

    def __getitem__(self, item):
        """
        :param item:
        :return: haze_img, ori_img
        """
        ori_image_name, haze_image_name = self.file_list[item]
        ori_image = self.transforms(Image.open(ori_image_name))
        haze_image = self.transforms(Image.open(haze_image_name))
        return ori_image, haze_image

    def __len__(self):
        return len(self.file_list)

    def get_image_pair_list(self):
        for image in self.image_name_list:
            image = image.split("/")[-1]
            key = image.split("_")[0] + "_" + image.split("_")[1] + ".jpg"
            if key in self.matching_dict.keys():
                self.matching_dict[key].append(image)
            else:
                self.matching_dict[key] = []
                self.matching_dict[key].append(image)

        for key in list(self.matching_dict.keys()):
            for hazy_image in self.matching_dict[key]:
                self.file_list.append([os.path.join(self.ori_root, key), os.path.join(self.haze_root, hazy_image)])

        random.shuffle(self.file_list)


In [ ]:
import argparse
# argument parser instead of command line
args = argparse.Namespace(
                epochs = 10,
                net_name = 'aod-xavier',
                lr = 1e-4,
                use_gpu = True,
                gpu = 1, #3
                ori_data_path = '/content/drive/MyDrive/AOD/original_image/image/',
                haze_data_path = '/content/drive/MyDrive/AOD/training_images/data/',
                val_ori_data_path = '/content/drive/MyDrive/AOD/original_image/image/',
                val_haze_data_path = '/content/drive/MyDrive/AOD/training_images/data/',
                num_workers = 2,
                batch_size = 8,
                val_batch_size = 16,
                print_gap = 500,
                model_dir = '/content/drive/MyDrive/AOD/models/',
                log_dir = '/content/drive/MyDrive/AOD/logs',
                sample_output_folder = '/content/samples',
                weight_decay = 0.0001,
                grad_clip_norm = 0.1,
                pretrain = False)

In [ ]:
import os
import torch
import torch.backends.cudnn
import torch.nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from tensorboardX import SummaryWriter
# from utils import logger, weight_init
# from config import get_config
# from model import AODnet
# from data import HazeDataset
pretrain_directory = '/content/drive/MyDrive/AOD/models/aod-xavier/AOD_1.pkl'

@logger
def load_data(cfg):
    data_transform = transforms.Compose([
        transforms.Resize([480, 640]),
        transforms.ToTensor()
    ])
    train_haze_dataset = HazeDataset(cfg.ori_data_path, cfg.haze_data_path, data_transform)
    train_loader = torch.utils.data.DataLoader(train_haze_dataset, batch_size=cfg.batch_size, shuffle=True,
                                               num_workers=cfg.num_workers, drop_last=True, pin_memory=True)

    val_haze_dataset = HazeDataset(cfg.val_ori_data_path, cfg.val_haze_data_path, data_transform)
    val_loader = torch.utils.data.DataLoader(val_haze_dataset, batch_size=cfg.val_batch_size, shuffle=False,
                                             num_workers=cfg.num_workers, drop_last=True, pin_memory=True)

    return train_loader, len(train_loader), val_loader, len(val_loader)


@logger
def save_model(epoch, path, net, optimizer, net_name):
    if not os.path.exists(os.path.join(path, net_name)):
        os.mkdir(os.path.join(path, net_name))
    torch.save({'epoch': epoch, 'state_dict': net.state_dict(), 'optimizer': optimizer.state_dict()},
               f=os.path.join(path, net_name, '{}_{}.pkl'.format('AOD', epoch)))


@logger
def load_network(device):
    net = AODnet().to(device)
    net.apply(weight_init)
    return net

@logger
def load_pretrain_network(device):
    net = AODnet().to(device)
    net.load_state_dict(torch.load(pretrain_directory)['state_dict'])
    return net

@logger
def load_optimizer(net, cfg):
    optimizer = torch.optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    return optimizer


@logger
def loss_func(device):
    criterion = torch.nn.MSELoss().to(device)
    return criterion


@logger
def load_summaries(cfg):
    summary = SummaryWriter(log_dir=os.path.join(cfg.log_dir, cfg.net_name), comment='')
    return summary


def main(cfg):
    # -------------------------------------------------------------------
    # basic config
    print(cfg)
    if cfg.gpu > -1:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(cfg.gpu)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # -------------------------------------------------------------------
    # load summaries
    summary = load_summaries(cfg)
    # -------------------------------------------------------------------
    # load data
    train_loader, train_number, val_loader, val_number = load_data(cfg)
    # -------------------------------------------------------------------
    # load loss
    criterion = loss_func(device)
    # -------------------------------------------------------------------
    # load network
    if cfg.pretrain == True:
      network = load_pretrain_network(device)
    else:
      network = load_network(device)
    # -------------------------------------------------------------------
    # load optimizer
    optimizer = load_optimizer(network, cfg)
    # -------------------------------------------------------------------
    # start train
    print('Start train')
    print(cfg.pretrain)
    network.train()
    if cfg.pretrain == True:
      current_epoch = pretrain_directory.split('_')
      current_epoch = current_epoch[1].split('.')
      current_epoch = int (current_epoch[0])+1
      for epoch in range(current_epoch,cfg.epochs):
        for step, (ori_image, haze_image) in enumerate(train_loader):
            count = epoch * train_number + (step + 1)
            ori_image, haze_image = ori_image.to(device), haze_image.to(device)
            dehaze_image = network(haze_image)
            loss = criterion(dehaze_image, ori_image)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(network.parameters(), cfg.grad_clip_norm)
            optimizer.step()
            summary.add_scalar('loss', loss.item(), count)
            if step % cfg.print_gap == 0:
                summary.add_image('DeHaze_Images', make_grid(dehaze_image[:4].data, normalize=True, scale_each=True),
                                  count)
                summary.add_image('Haze_Images', make_grid(haze_image[:4].data, normalize=True, scale_each=True), count)
                summary.add_image('Origin_Images', make_grid(ori_image[:4].data, normalize=True, scale_each=True),
                                  count)
            print('Epoch: {}/{}  |  Step: {}/{}  |  lr: {:.6f}  | Loss: {:.6f}'
                  .format(epoch + 1, cfg.epochs, step + 1, train_number,
                          optimizer.param_groups[0]['lr'], loss.item()))
        # -------------------------------------------------------------------
        # start validation
        print('Epoch: {}/{} | Validation Model Saving Images'.format(epoch + 1, cfg.epochs))
        network.eval()
        for step, (ori_image, haze_image) in enumerate(val_loader):
            if step > 10:   # only save image 10 times
                break
            ori_image, haze_image = ori_image.to(device), haze_image.to(device)
            dehaze_image = network(haze_image)
            torchvision.utils.save_image(
                torchvision.utils.make_grid(torch.cat((haze_image, dehaze_image, ori_image), 0),
                                            nrow=ori_image.shape[0]),
                os.path.join(cfg.sample_output_folder, '{}_{}.jpg'.format(epoch + 1, step)))
        network.train()
        # -------------------------------------------------------------------
        # save per epochs model
        save_model(epoch, cfg.model_dir, network, optimizer, cfg.net_name)
        # -------------------------------------------------------------------
         # train finish
        summary.close()
    else:
      for epoch in range(cfg.epochs):
          for step, (ori_image, haze_image) in enumerate(train_loader):
              count = epoch * train_number + (step + 1)
              ori_image, haze_image = ori_image.to(device), haze_image.to(device)
              dehaze_image = network(haze_image)
              loss = criterion(dehaze_image, ori_image)
              optimizer.zero_grad()
              loss.backward()
              torch.nn.utils.clip_grad_norm_(network.parameters(), cfg.grad_clip_norm)
              optimizer.step()
              summary.add_scalar('loss', loss.item(), count)
              if step % cfg.print_gap == 0:
                  summary.add_image('DeHaze_Images', make_grid(dehaze_image[:4].data, normalize=True, scale_each=True),
                                    count)
                  summary.add_image('Haze_Images', make_grid(haze_image[:4].data, normalize=True, scale_each=True), count)
                  summary.add_image('Origin_Images', make_grid(ori_image[:4].data, normalize=True, scale_each=True),
                                    count)
              print('Epoch: {}/{}  |  Step: {}/{}  |  lr: {:.6f}  | Loss: {:.6f}'
                    .format(epoch + 1, cfg.epochs, step + 1, train_number,
                            optimizer.param_groups[0]['lr'], loss.item()))
          # -------------------------------------------------------------------
          # start validation
          print('Epoch: {}/{} | Validation Model Saving Images'.format(epoch + 1, cfg.epochs))
          network.eval()
          for step, (ori_image, haze_image) in enumerate(val_loader):
              if step > 10:   # only save image 10 times
                  break
              ori_image, haze_image = ori_image.to(device), haze_image.to(device)
              dehaze_image = network(haze_image)
              torchvision.utils.save_image(
                  torchvision.utils.make_grid(torch.cat((haze_image, dehaze_image, ori_image), 0),
                                              nrow=ori_image.shape[0]),
                  os.path.join(cfg.sample_output_folder, '{}_{}.jpg'.format(epoch + 1, step)))
          network.train()
          # -------------------------------------------------------------------
          # save per epochs model
          save_model(epoch, cfg.model_dir, network, optimizer, cfg.net_name)
      # -------------------------------------------------------------------
      # train finish
      summary.close()


if __name__ == '__main__':
    # config_args, unparsed_args = args.
    main(args)


2021-08-20 09:37:55,175 - load_summaries - INFO - Start running load_summaries


Namespace(batch_size=8, epochs=10, gpu=1, grad_clip_norm=0.1, haze_data_path='/content/drive/MyDrive/AOD/training_images/data/', log_dir='/content/drive/MyDrive/AOD/logs', lr=0.0001, model_dir='/content/drive/MyDrive/AOD/models/', net_name='aod-xavier', num_workers=2, ori_data_path='/content/drive/MyDrive/AOD/original_image/image/', pretrain=False, print_gap=500, sample_output_folder='/content/samples', use_gpu=True, val_batch_size=16, val_haze_data_path='/content/drive/MyDrive/AOD/training_images/data/', val_ori_data_path='/content/drive/MyDrive/AOD/original_image/image/', weight_decay=0.0001)


2021-08-20 09:37:55,627 - root - INFO - Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
2021-08-20 09:37:55,658 - root - INFO - Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
2021-08-20 09:37:55,812 - load_summaries - INFO - Done running load_summaries
2021-08-20 09:37:55,814 - load_data - INFO - Start running load_data


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# # net = load_network(device)

# net = AODnet().to(device)
# net1= net.load_state_dict(torch.load('/content/AOD-Net-PyTorch/aod-xavier/AOD_0.pkl')['state_dict'])
pretrain_directory = '/content/drive/MyDrive/AOD/models/aod-xavier/AOD_5.pkl'
@logger
def load_pretrain_network(device):
    net = AODnet().to(device)
    net.load_state_dict(torch.load(pretrain_directory)['state_dict'])
    return net
network = load_pretrain_network(device)

2021-08-12 11:10:16,962 - load_pretrain_network - INFO - Start running load_pretrain_network
2021-08-12 11:10:29,442 - load_pretrain_network - INFO - Done running load_pretrain_network


In [ ]:
import torchvision
def make_test_data(img_path_list, device):
    data_transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize([480, 640]),
        torchvision.transforms.ToTensor()
    ])
    imgs = []
    for img_path in img_path_list:
        x = data_transform(Image.open(img_path)).unsqueeze(0)
        x = x.to(device)
        imgs.append(x)
    return imgs

In [ ]:
!mkdir results

In [ ]:
import numpy as np
from PIL import Image
import glob
import matplotlib.pyplot as plt

# image = make_test_data(glob.glob('/content/drive/MyDrive/AOD/test/*.jpg'),device)
# image = make_test_data(glob.glob('/content/drive/MyDrive/AOD/fog/*.jpg'),device)
image = make_test_data(glob.glob('/content/drive/MyDrive/pku-autonomous-driving/train_images/ID_0a3c2c009.jpg'),device)
# image = make_test_data(glob.glob('/content/drive/MyDrive/AOD/original_image/image/*.jpg'),device)
network.eval()
for y,i in enumerate(image):
  try:
    dehaze_image = network(i)
    #********************** Uncomment to get side by side comparison *******************************
    # torchvision.utils.save_image(torch.cat((i, dehaze_image), 0), "results/res"+str (y)+".jpg")

    #********************** Uncomment to get only dehazed images ***********************************
    torchvision.utils.save_image(i, "results/original"+str (y)+".jpg")
    torchvision.utils.save_image(dehaze_image, "results/dehazed"+str (y)+".jpg")
  except:
    print(y)

In [ ]:
import os
for subdir, dirs, files in os.walk('/content/results'):
    print(len(files))
    total_no_images = len(files)

598


In [ ]:
# from skimage.measure import structural_similarity as ssim
from skimage import measure
import cv2
ssim = 0
for i in range(int ((total_no_images/2)+1)):
  try:
    print(i)
    original = np.array(Image.open('/content/results/original'+str (i)+'.jpg'))
    dehazed = np.array(Image.open('/content/results/dehazed'+str (i)+'.jpg'))
    ssim += measure.compare_ssim(original,dehazed, multichannel=True)
  except:
    print(i)
print(ssim/(total_no_images/2))

0
1
2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  # Remove the CWD from sys.path while we load stuff.


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
original = np.array(Image.open('/content/results/original0.jpg'))
dehazed = np.array(Image.open('/content/results/dehazed0.jpg'))
# plt.imshow(dehazed)
measure.compare_ssim(original,dehazed, multichannel=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  after removing the cwd from sys.path.


0.8481487202536986

In [ ]:
from math import log10, sqrt
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [ ]:
psnr = 0
for i in range(int ((total_no_images/2)+1)):
  try:
    print(i)
    original = np.array(Image.open('/content/results/original'+str (i)+'.jpg'))
    dehazed = np.array(Image.open('/content/results/dehazed'+str (i)+'.jpg'))
    psnr += PSNR(original,dehazed)
  except:
    print(i)
print(psnr/(total_no_images/2))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [ ]:
original = np.array(Image.open('/content/results/original0.jpg'))
dehazed = np.array(Image.open('/content/results/dehazed0'+'.jpg'))
psnr = PSNR(original,dehazed)
print(psnr)
from skimage import measure
ssim = measure.compare_ssim(original,dehazed, multichannel=True)
print(ssim)

27.728293393725476
0.8481487202536986


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  


In [ ]:
mse = 0
for i in range(int ((total_no_images/2)+1)):
  try:
    print(i)
    original = np.array(Image.open('/content/results/original'+str (i)+'.jpg'))
    dehazed = np.array(Image.open('/content/results/dehazed'+str (i)+'.jpg'))
    mse += np.mean((original - dehazed) ** 2)
  except:
    print(i)
print(mse/(total_no_images/2))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [ ]:
mse = np.mean((original - dehazed) ** 2)
print(mse)

109.71121419270834


In [ ]:
plt.imshow(original)

In [ ]:
plt.imshow(dehazed)

NameError: ignored

In [ ]:
import shutil
shutil.make_archive('results', 'zip', '/content/results')